In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

# %matplotlib notebook
%matplotlib inline

In [2]:
def RungeKutta(ddy, dy0, y0, x, h):
    y = [0 for i in range(len(x))]
    z = [0 for i in range(len(x))]
    y[0] = y0
    z[0] = dy0

    for i in range(1, len(x)):
        #   коэфиценты для dy/dx = z
        ky1 = z[i - 1]
        ky2 = z[i - 1] + h * ky1 / 2
        ky3 = z[i - 1] + h * ky2 / 2
        ky4 = z[i - 1] + h * ky3

        #   коэфиценты для dz/dx = f(x, y, z)
        kz1 = ddy(x[i - 1], y[i - 1], z[i - 1])
        kz2 = ddy(x[i - 1] + h / 2, y[i - 1] + h * kz1 / 2, z[i - 1] + h * kz1 / 2)
        kz3 = ddy(x[i - 1] + h / 2, y[i - 1] + h * kz2 / 2, z[i - 1] + h * kz2 / 2)
        kz4 = ddy(x[i - 1] + h, y[i - 1] + h * kz3, z[i - 1] + h * kz3)

        z[i] = z[i - 1] + h * (kz1 + 2 * kz2 + 2 * kz3 + kz4) / 6
        y[i] = y[i - 1] + h * (ky1 + 2 * ky2 + 2 * ky3 + ky4) / 6

    return y


def shooting(y, ddy, y1, y2, x, h, eps=0.0001):
    n0 = y2
    n1 = 1.8
    n = [n0, n1]
    yn1 = RungeKutta(ddy, n0, y1, x, h)
    yn2 = RungeKutta(ddy, n1, y1, x, h)
    yn = [yn1, yn2]
    
    j = 0
    while abs(yn[-1] - y2) > eps:
        nk = n[j+1] - (n[j+1] - n[j])/(yn[j+1][-1] - yn[j][-1]) * yn[j+1][-1]
        yn.append(nk)
        n.append(nk)
        j += 1
    

In [3]:
function = lambda x, y, dy: 2*y/(x*x*(x+1))
interval = [1, 2]
step = 0.1
y1 = 1 + 4 * math.log(2)
y2 = -1 + 3 * math.log(2)
# Точное решение
y_lambda = lambda x: -1 + 2/x + 2*(x + 1)/x * math.log(abs(x + 1))